# Lord Of The Rings PyRaphtory Example Notebook 🧝🏻‍♀️🧙🏻‍♂️💍

## Setup environment and download data 💾

Import all necessary dependencies needed to build a graph from your data in PyRaphtory. Download csv data from github into your tmp folder (file path: /tmp/lotr.csv).

In [6]:
from pathlib import Path
from pyraphtory.context import PyRaphtory
from pyraphtory.vertex import Vertex
from pyraphtory.spouts import FileSpout
from pyraphtory.builder import *
from pyvis.network import Network
import csv

!curl -o /tmp/lotr.csv https://raw.githubusercontent.com/Raphtory/Data/main/lotr.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 52206  100 52206    0     0   684k      0 --:--:-- --:--:-- --:--:--  738k


## Preview data  👀

Preview the head of the dataset.

In [2]:
!head /tmp/lotr.csv

Gandalf,Elrond,33
Frodo,Bilbo,114
Blanco,Marcho,146
Frodo,Bilbo,205
Thorin,Gandalf,270
Thorin,Bilbo,270
Gandalf,Bilbo,270
Gollum,Bilbo,286
Gollum,Bilbo,306
Gollum,Bilbo,308


In [3]:
filename = '/tmp/lotr.csv'

## Create a new Raphtory graph 📊

Turn on logs to see what is going on in PyRaphtory. Initialise Raphtory by creating a PyRaphtory object. Create your new graph.

In [4]:
pr = PyRaphtory(logging=True).open()
rg = pr.new_graph()

16:10:49.846 [io-compute-4] INFO  com.raphtory.internals.management.Py4JServer - Starting PythonGatewayServer...
Port: 56598
Secret: 6d551383dffa40aabc1a78b66856208b73ede3b8a387d66069dd404ec41f172d


16:10:50.418 [Thread-12] INFO  com.raphtory.internals.context.LocalContext$ - Creating Service for 'hungry_lime_python'
16:10:50.432 [io-compute-6] INFO  com.raphtory.internals.management.Prometheus$ - Prometheus started on port /0:0:0:0:0:0:0:0:9999
16:10:51.005 [io-compute-6] INFO  com.raphtory.internals.components.partition.PartitionOrchestrator$ - Creating '1' Partition Managers for 'hungry_lime_python'.
16:10:53.021 [io-compute-1] INFO  com.raphtory.internals.components.partition.PartitionManager - Partition 0: Starting partition manager for 'hungry_lime_python'.


## Ingest the data into a graph 😋

Write a parsing method to parse your csv file and ultimately create a graph.

In [32]:
with open(filename, 'r') as csvfile:
    datareader = csv.reader(csvfile)
    for row in datareader:
        source_node = row[0]
        src_id = rg.assign_id(source_node)
        target_node = row[1]
        tar_id = rg.assign_id(target_node)
        time_stamp = int(row[2])
        rg.add_vertex(time_stamp, src_id, Properties(ImmutableProperty("name", source_node)), Type("Character"))
        rg.add_vertex(time_stamp, tar_id, Properties(ImmutableProperty("name", target_node)), Type("Character"))
        rg.add_edge(time_stamp, src_id, tar_id, Type("Character_Co-occurence"))

## Collect simple metrics 📈

Select certain metrics to show in your output dataframe. Here we have selected vertex name, degree, out degree and in degree. 

In [33]:
from pyraphtory.graph import Row
df = rg \
      .select(lambda vertex: Row(vertex.name(), vertex.degree(), vertex.out_degree(), vertex.in_degree())) \
      .write_to_dataframe(["name", "degree", "out_degree", "in_degree"])

16:32:37.105 [spawner-akka.actor.default-dispatcher-5] INFO  com.raphtory.internals.components.querymanager.QueryManager - Source '0' is unblocking analysis for Graph 'hungry_lime_python' with 7959 messages sent.
16:32:37.107 [io-compute-6] INFO  com.raphtory.api.querytracker.QueryProgressTracker - Job 536385567_6698200171266357587: Starting query progress tracker.
16:32:37.112 [spawner-akka.actor.default-dispatcher-5] INFO  com.raphtory.internals.components.querymanager.QueryManager - Query '536385567_6698200171266357587' received, your job ID is '536385567_6698200171266357587'.
16:32:37.120 [spawner-akka.actor.default-dispatcher-9] INFO  com.raphtory.internals.components.partition.QueryExecutor - 536385567_6698200171266357587_0: Starting QueryExecutor.
16:32:37.873 [spawner-akka.actor.default-dispatcher-10] INFO  com.raphtory.internals.components.querymanager.QueryHandler - Job '536385567_6698200171266357587': Perspective at Time '32674' took 744 ms to run. 
16:32:37.873 [spawner-akk

**Clean the dataframe, we have deleted the unused window column.** 🧹

In [35]:
## clean
df.drop(columns=['window'], inplace=True)

### Preview the dataframe  👀

In [36]:
df

,timestamp,name,degree,out_degree,in_degree
0,32674,Hirgon,2,2,0
1,32674,Hador,3,1,2
2,32674,Horn,4,1,3
3,32674,Galadriel,19,6,16
4,32674,Isildur,18,18,0
...,...,...,...,...,...
134,32674,Faramir,29,3,29
135,32674,Bain,2,1,1
136,32674,Walda,13,3,10
137,32674,Thranduil,2,0,2


**Sort by highest degree, top 10**

In [8]:
df.sort_values(['degree'], ascending=False)[:10]

,timestamp,name,degree,out_degree,in_degree
55,32674,Frodo,51,37,22
54,32674,Gandalf,49,35,24
97,32674,Aragorn,45,5,45
63,32674,Merry,34,23,18
32,32674,Pippin,34,30,10
56,32674,Elrond,32,18,24
52,32674,Théoden,30,22,9
134,32674,Faramir,29,3,29
118,32674,Sam,28,20,17
129,32674,Gimli,25,22,11


**Sort by highest in-degree, top 10**

In [9]:
df.sort_values(['in_degree'], ascending=False)[:10]

,timestamp,name,degree,out_degree,in_degree
97,32674,Aragorn,45,5,45
134,32674,Faramir,29,3,29
54,32674,Gandalf,49,35,24
56,32674,Elrond,32,18,24
55,32674,Frodo,51,37,22
63,32674,Merry,34,23,18
138,32674,Boromir,18,6,17
118,32674,Sam,28,20,17
3,32674,Galadriel,19,6,16
132,32674,Legolas,25,18,16


**Sort by highest out-degree, top 10**

In [10]:
df.sort_values(['out_degree'], ascending=False)[:10]

,timestamp,name,degree,out_degree,in_degree
55,32674,Frodo,51,37,22
54,32674,Gandalf,49,35,24
32,32674,Pippin,34,30,10
63,32674,Merry,34,23,18
52,32674,Théoden,30,22,9
129,32674,Gimli,25,22,11
118,32674,Sam,28,20,17
56,32674,Elrond,32,18,24
4,32674,Isildur,18,18,0
132,32674,Legolas,25,18,16


# Run a PageRank algorithm 📑

Run your selected algorithm on your graph, here we run PageRank. Your algorithms can be obtained from the PyRaphtory object you created at the start. Specify where you write the result of your algorithm to, e.g. the additional column results in your dataframe.

In [11]:
cols = ["prlabel"]

df_pagerank = rg.at(32674) \
                .past() \
                .transform(pr.algorithms.generic.centrality.PageRank())\
                .execute(pr.algorithms.generic.NodeList(*cols)) \
                .write_to_dataframe(["name"] + cols)

17:00:42.750 [io-compute-3] INFO  com.raphtory.api.querytracker.QueryProgressTracker - Job PageRank:NodeList_8283484491969709751: Starting query progress tracker.
17:00:42.766 [spawner-akka.actor.default-dispatcher-5] INFO  com.raphtory.internals.components.querymanager.QueryManager - Query 'PageRank:NodeList_8283484491969709751' received, your job ID is 'PageRank:NodeList_8283484491969709751'.
17:00:42.767 [spawner-akka.actor.default-dispatcher-9] INFO  com.raphtory.internals.components.partition.QueryExecutor - PageRank:NodeList_8283484491969709751_0: Starting QueryExecutor.
17:00:42.934 [spawner-akka.actor.default-dispatcher-7] INFO  com.raphtory.internals.components.querymanager.QueryHandler - Job 'PageRank:NodeList_8283484491969709751': Perspective at Time '32674' took 165 ms to run. 
17:00:42.934 [spawner-akka.actor.default-dispatcher-8] INFO  com.raphtory.api.querytracker.QueryProgressTracker - Job 'PageRank:NodeList_8283484491969709751': Perspective '32674' finished in 184 ms.


**Clean your dataframe** 🧹

In [12]:
## clean
df_pagerank.drop(columns=['window'], inplace=True)

In [13]:
df_pagerank

,timestamp,name,prlabel
0,32674,Hirgon,0.277968
1,32674,Hador,0.459710
2,32674,Horn,0.522389
3,32674,Galadriel,2.228852
4,32674,Isildur,0.277968
...,...,...,...
134,32674,Faramir,8.551166
135,32674,Bain,0.396105
136,32674,Walda,0.817198
137,32674,Thranduil,0.761719


**The top ten most ranked**

In [14]:
df_pagerank.sort_values(['prlabel'], ascending=False)[:10]

,timestamp,name,prlabel
97,32674,Aragorn,13.246457
134,32674,Faramir,8.551166
56,32674,Elrond,5.621548
138,32674,Boromir,4.824014
132,32674,Legolas,4.622590
110,32674,Imrahil,4.095600
65,32674,Éomer,3.473897
42,32674,Samwise,3.292762
118,32674,Sam,2.826140
55,32674,Frodo,2.806475


## Run a connected components algorithm 

Example running connected components algorithm on the graph.

In [15]:
cols = ["cclabel"]
df_cc = rg.at(32674) \
                .past() \
                .transform(pr.algorithms.generic.ConnectedComponents)\
                .execute(pr.algorithms.generic.NodeList(*cols)) \
                .write_to_dataframe(["name"] + cols)

17:00:47.221 [io-compute-6] INFO  com.raphtory.api.querytracker.QueryProgressTracker - Job ConnectedComponents:NodeList_7963507704994007119: Starting query progress tracker.
17:00:47.225 [spawner-akka.actor.default-dispatcher-8] INFO  com.raphtory.internals.components.querymanager.QueryManager - Query 'ConnectedComponents:NodeList_7963507704994007119' received, your job ID is 'ConnectedComponents:NodeList_7963507704994007119'.
17:00:47.226 [spawner-akka.actor.default-dispatcher-9] INFO  com.raphtory.internals.components.partition.QueryExecutor - ConnectedComponents:NodeList_7963507704994007119_0: Starting QueryExecutor.
17:00:47.259 [spawner-akka.actor.default-dispatcher-5] INFO  com.raphtory.internals.components.querymanager.QueryHandler - Job 'ConnectedComponents:NodeList_7963507704994007119': Perspective at Time '32674' took 31 ms to run. 
17:00:47.259 [spawner-akka.actor.default-dispatcher-11] INFO  com.raphtory.api.querytracker.QueryProgressTracker - Job 'ConnectedComponents:NodeL

**Clean dataframe.**

In [16]:
## clean
df_cc.drop(columns=['window'], inplace=True)

**Preview dataframe.**

In [17]:
df_cc

,timestamp,name,cclabel
0,32674,Hirgon,-8637342647242242534
1,32674,Hador,-8637342647242242534
2,32674,Horn,-8637342647242242534
3,32674,Galadriel,-8637342647242242534
4,32674,Isildur,-8637342647242242534
...,...,...,...
134,32674,Faramir,-8637342647242242534
135,32674,Bain,-6628080393138316116
136,32674,Walda,-8637342647242242534
137,32674,Thranduil,-8637342647242242534


### Number of distinct components 

Extract number of distinct components, which is 3 in this dataframe.

In [18]:
len(set(df_cc['cclabel']))

3

### Size of components 

Calculate the size of the 3 connected components.

In [19]:
df_cc.groupby(['cclabel']).count().reset_index().drop(columns=['timestamp'])

,cclabel,name
0,-8637342647242242534,134
1,-6628080393138316116,3
2,-5499479516525190226,2


### Run chained algorithms at once 

In this example, we chain PageRank, Connected Components and Degree algorithms, running them one after another on the graph. Specify all the columns in the output dataframe, including an output column for each algorithm in the chain.

In [20]:
cols = ["inDegree", "outDegree", "degree","prlabel","cclabel"]

df_chained = rg.at(32674) \
                .past() \
                .transform(pr.algorithms.generic.centrality.PageRank())\
                .transform(pr.algorithms.generic.ConnectedComponents)\
                .transform(pr.algorithms.generic.centrality.Degree())\
                .execute(pr.algorithms.generic.NodeList(*cols)) \
                .write_to_dataframe(["name"] + cols)

17:00:51.445 [io-compute-9] INFO  com.raphtory.api.querytracker.QueryProgressTracker - Job PageRank:ConnectedComponents:Degree:NodeList_3188262466645783136: Starting query progress tracker.
17:00:51.449 [spawner-akka.actor.default-dispatcher-9] INFO  com.raphtory.internals.components.querymanager.QueryManager - Query 'PageRank:ConnectedComponents:Degree:NodeList_3188262466645783136' received, your job ID is 'PageRank:ConnectedComponents:Degree:NodeList_3188262466645783136'.
17:00:51.450 [spawner-akka.actor.default-dispatcher-6] INFO  com.raphtory.internals.components.partition.QueryExecutor - PageRank:ConnectedComponents:Degree:NodeList_3188262466645783136_0: Starting QueryExecutor.
17:00:51.521 [spawner-akka.actor.default-dispatcher-7] INFO  com.raphtory.api.querytracker.QueryProgressTracker - Job 'PageRank:ConnectedComponents:Degree:NodeList_3188262466645783136': Perspective '32674' finished in 76 ms.
17:00:51.521 [spawner-akka.actor.default-dispatcher-9] INFO  com.raphtory.internals

In [2]:
df_chained.drop(columns=['window'], inplace=True)

NameError: name 'df_chained' is not defined

In [22]:
df_chained

,timestamp,window,name,inDegree,outDegree,degree,prlabel,cclabel
0,32674,None,Hirgon,0,2,2,0.277968,-8637342647242242534
1,32674,None,Hador,2,1,3,0.459710,-8637342647242242534
2,32674,None,Horn,3,1,4,0.522389,-8637342647242242534
3,32674,None,Galadriel,16,6,19,2.228852,-8637342647242242534
4,32674,None,Isildur,0,18,18,0.277968,-8637342647242242534
...,...,...,...,...,...,...,...,...
134,32674,None,Faramir,29,3,29,8.551166,-8637342647242242534
135,32674,None,Bain,1,1,2,0.396105,-6628080393138316116
136,32674,None,Walda,10,3,13,0.817198,-8637342647242242534
137,32674,None,Thranduil,2,0,2,0.761719,-8637342647242242534


### Create visualisation by adding nodes 🔎

In [23]:
def visualise(rg, df_chained):
    # Create network object
    net = Network(notebook=True, height='750px', width='100%', bgcolor='#222222', font_color='white')
    # Set visualisation tool
    net.force_atlas_2based()
    # Get the node list 
    df_node_list = rg.at(32674) \
                .past() \
                .execute(pr.algorithms.generic.NodeList()) \
                .write_to_dataframe(['name'])
    
    nodes = df_node_list['name'].tolist()
    
    node_data = []
    ignore_items = ['timestamp', 'name', 'window']
    for node_name in nodes:
        for i, row in df_chained.iterrows():
            if row['name']==node_name:
                data = ''
                for k,v in row.iteritems():
                    if k not in ignore_items:
                        data = data+str(k)+': '+str(v)+'\n'
                node_data.append(data)
                continue
    # Add the nodes
    net.add_nodes(nodes, title=node_data)
    # Get the edge list
    df_edge_list = rg.at(32674) \
            .past() \
            .execute(pr.algorithms.generic.EdgeList()) \
            .write_to_dataframe(['from', 'to'])
    edges = []
    for i, row in df_edge_list[['from', 'to']].iterrows():
        edges.append([row['from'], row['to']])
    # Add the edges
    net.add_edges(edges)
    # Toggle physics
    net.toggle_physics(True)
    return net

In [24]:
net = visualise(rg, df_chained)

17:00:52.169 [io-compute-3] INFO  com.raphtory.api.querytracker.QueryProgressTracker - Job NodeList_3782182765496279863: Starting query progress tracker.
17:00:52.169 [spawner-akka.actor.default-dispatcher-7] INFO  com.raphtory.internals.components.querymanager.QueryManager - Query 'NodeList_3782182765496279863' received, your job ID is 'NodeList_3782182765496279863'.
17:00:52.170 [spawner-akka.actor.default-dispatcher-7] INFO  com.raphtory.internals.components.partition.QueryExecutor - NodeList_3782182765496279863_0: Starting QueryExecutor.
17:00:52.176 [spawner-akka.actor.default-dispatcher-3] INFO  com.raphtory.internals.components.querymanager.QueryHandler - Job 'NodeList_3782182765496279863': Perspective at Time '32674' took 4 ms to run. 
17:00:52.176 [spawner-akka.actor.default-dispatcher-11] INFO  com.raphtory.api.querytracker.QueryProgressTracker - Job 'NodeList_3782182765496279863': Perspective '32674' finished in 7 ms.
17:00:52.176 [spawner-akka.actor.default-dispatcher-11] I

## Show the html file of the visualisation

In [25]:
net.show('preview.html')

## Shut down PyRaphtory  🛑

In [26]:
pr.shutdown()

17:00:52.589 [Thread-12] INFO  com.raphtory.internals.management.PythonInterop$ - Shutting down pyraphtory
